In [1]:
# config_loader.py

import sqlite3

class ConfigLoader:
    def __init__(self, db_path='../config/config.db'):
        self.db_path = db_path

    def load(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        config_data = {}

        cursor.execute("SELECT id, name FROM model_providers")
        providers = cursor.fetchall()

        for provider_id, provider_name in providers:
            config_data[provider_name] = {}

            cursor.execute("SELECT id, model_name, model_value FROM models WHERE provider_id=?", (provider_id,))
            models = cursor.fetchall()

            for model_id, model_name, model_value in models:
                cursor.execute("SELECT base_url, api_key, enable FROM model_apis WHERE model_id=? AND enable=1", (model_id,))
                apis = cursor.fetchall()
                api_list = [{"base_url": b, "api_key": k} for b, k, e in apis]

                config_data[provider_name][model_name] = {
                    "model_value": model_value,
                    "apis": api_list
                }
        conn.close()
        return config_data

if __name__ == "__main__":
    loader = ConfigLoader()
    config = loader.load()
    print(config)


{'baidu': {'ERNIE-Lite-8K': {'model_value': 'ernie-lite-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Speed-128K': {'model_value': 'ernie-speed-128k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Speed-8K': {'model_value': 'ernie-speed-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Tiny-8K': {'model_value': 'ernie-tiny-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}}, 'google': {'gemini-2.0-flash': {'model_value': 'gemini-2.0-flash', 'apis': [{'base_url': 'https://api.hunyuan.cloud.tencent.com/v1', 'api_key': 'AIzaSyBq7U8IxF6lUGWpN4SdqLCojB2Fo-Z

In [3]:


loader = ConfigLoader()
config = loader.load()
# 直接用config获取所有配置信息
print(config)

{'baidu': {'ERNIE-Lite-8K': {'model_value': 'ernie-lite-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Speed-128K': {'model_value': 'ernie-speed-128k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Speed-8K': {'model_value': 'ernie-speed-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}, 'ERNIE-Tiny-8K': {'model_value': 'ernie-tiny-8k', 'apis': [{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}]}}, 'google': {'gemini-2.0-flash': {'model_value': 'gemini-2.0-flash', 'apis': [{'base_url': 'https://api.hunyuan.cloud.tencent.com/v1', 'api_key': 'AIzaSyBq7U8IxF6lUGWpN4SdqLCojB2Fo-Z

In [3]:
import sqlite3
import json

class ConfigLoader:
    def __init__(self, db_path='../config/config.db'):
        self.db_path = db_path

    def get_apis_and_provider_by_model_value(self, model_value):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute("SELECT id, provider_id FROM models WHERE model_value=?", (model_value,))
        row = cursor.fetchone()
        if not row:
            conn.close()
            return None

        model_id, provider_id = row

        cursor.execute("SELECT name FROM model_providers WHERE id=?", (provider_id,))
        provider_row = cursor.fetchone()
        provider_name = provider_row[0] if provider_row else None

        cursor.execute("SELECT base_url, api_key FROM model_apis WHERE model_id=? AND enable=1", (model_id,))
        apis = cursor.fetchall()
        conn.close()

        api_list = [{"base_url": base_url, "api_key": api_key} for base_url, api_key in apis]

        return {
            "provider_name": provider_name,
            "apis": api_list
        }

    def get_all_models(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute("""
            SELECT m.model_name, m.model_value, p.name
            FROM models m
            JOIN model_providers p ON m.provider_id = p.id
        """)
        rows = cursor.fetchall()
        conn.close()

        result = {}
        for model_name, model_value, provider_name in rows:
            result[model_name] = {
                "model_value": model_value,
                "provider_name": provider_name
            }
        return result

if __name__ == "__main__":
    loader = ConfigLoader()

    # 查询单个模型的api和厂商
    result = loader.get_apis_and_provider_by_model_value("ernie-lite-8k")
    if result:
        print("单模型查询结果：")
        print(json.dumps(result, indent=2, ensure_ascii=False))
    else:
        print("未找到对应模型或API")

    # 查询所有模型及厂商
    all_models = loader.get_all_models()
    print("\n所有模型信息：")
    print(json.dumps(all_models, indent=2, ensure_ascii=False))


单模型查询结果：
{
  "provider_name": "baidu",
  "apis": [
    {
      "base_url": "https://qianfan.baidubce.com/v2",
      "api_key": "bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c"
    }
  ]
}

所有模型信息：
{
  "GLM-4.5-Flash": {
    "model_value": "GLM-4.5-Flash",
    "provider_name": "zhipu"
  },
  "GLM-4-Flash": {
    "model_value": "GLM-4-Flash-250414",
    "provider_name": "zhipu"
  },
  "GLM-Z1-Flash": {
    "model_value": "GLM-Z1-Flash",
    "provider_name": "zhipu"
  },
  "GLM-4.1V-Thinking-Flash": {
    "model_value": "GLM-4.1V-Thinking-Flash",
    "provider_name": "zhipu"
  },
  "GLM-4V-Flash": {
    "model_value": "GLM-4V-Flash",
    "provider_name": "zhipu"
  },
  "Cogview-3-Flash": {
    "model_value": "Cogview-3-Flash",
    "provider_name": "zhipu"
  },
  "CogVideoX-Flash": {
    "model_value": "CogVideoX-Flash",
    "provider_name": "zhipu"
  },
  "Spark-Lite": {
    "model_value": "Lite",
    "provider_name": "xunfei"
  },
  "ERNIE-Speed-128K": {
    

In [4]:
loader = ConfigLoader()
result = loader.get_apis_and_provider_by_model_value("ernie-lite-8k")
if result and "apis" in result and len(result["apis"]) > 0:
    first_api = result["apis"][0]
    print(first_api)

{'base_url': 'https://qianfan.baidubce.com/v2', 'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c'}


In [6]:
from utils import get_model_config
get_model_config("ernie-lite-8k")

{'api_key': 'bce-v3/ALTAK-4KyFUYBYZMuf8s3bxg2nO/5852d9dd9a49dccc9234f0907a1b53645456469c',
 'base_url': 'https://qianfan.baidubce.com/v2'}